In [1]:
# import libraries
import pandas as pd
import numpy as np

from pyvis.network import Network
import networkx as nx
import matplotlib.pyplot as plt

from collections import Counter
import colorsys

In [2]:
# import data from xlsx file
df = pd.read_excel('data/AI Partnerships.xlsx', sheet_name='data')

# rename 'source' column to 'partner1' and 'target' column to 'partner2'
df = df.rename(columns={'source': 'partner1', 'target': 'partner2', 'Type': 'type'})

In [4]:
# rename some of the labels
df['type'] = df['type'].replace('Build real life applications', 'Real-life apps')
df['type'] = df['type'].replace('Building a platform', 'Platform')
df['type'] = df['type'].replace('Distribution (not mere "sell-to" motion)', 'Distribution')

In [5]:
df[['partner1', 'partner2']] = df.apply(lambda row: pd.Series(sorted([row['partner1'], row['partner2']])), axis=1)

# group data by 'type', 'source' and 'target' columns and sum 'weight' column
df = df.groupby(['type','partner1', 'partner2']).sum().reset_index()

# create a list of all companies
all_companies = set(df['partner1'].tolist() + df['partner2'].tolist())

In [12]:
# Create a NetworkX graph
G = nx.Graph()

# Add edges to the graph
for _, row in df.iterrows():
    G.add_edge(row['partner1'], row['partner2'], type=row['type'])

# Calculate degree for each node
degrees = dict(G.degree())


# Normalize degrees to use as node sizes
max_degree = max(degrees.values())
min_size, max_size = 5, 100  # Min and max node sizes
normalized_degrees = {node: ((degree / max_degree) * (max_size - min_size) + min_size) 
                      for node, degree in degrees.items()}


# Create a Pyvis network
net = Network(notebook=True, directed=False ,width="100%", height="100%", font_color="#FAC13C", bgcolor="#1A1D24" ,cdn_resources='remote', select_menu=True, filter_menu=True)

# Define the list of companies to have the same color
special_companies = ['OpenAI', 'Anthropic', 'Cohere', 'Mistral', 'Microsoft', 'Meta', 'Perplexity', 'NVIDIA', 'Google', 'AWS', 'Snowflake', 'Databricks', 'Hugging Face', 'Scale AI']

# Define color map
color_map = {
    'AI Safety': '#10BFCC', # Teal
    'Real-life apps': '#A0C98B', # Green
    'Platform': '#DB6FBF', # Pink
    'Compute / AI infra': '#e8702a', # Yellow
    'Data - News outlets': '#2c852d', # Dark green
    'Data - Other': '#1F85DE', # Blue
    'Distribution': '#E46476', # Red
    'R&D': '#9E7FCC', # Purple
    'Primary nodes': '#FAC13C' #gold
}


# Get unique companies
unique_companies = set(df['partner1'].unique()) | set(df['partner2'].unique())

# Function to get company color
def get_company_color(company):
    if company in special_companies:
        return color_map['Primary nodes']
    company_types = df[(df['partner1'] == company) | (df['partner2'] == company)]['type'].unique()
    if len(company_types) > 0:
        return color_map.get(company_types[0], '#FFFFFF')  # White as default if type not in color_map
    return '#FFFFFF'  # White as default


# Add nodes to the network with size based on degree and color based on type
for company in unique_companies:
    size = normalized_degrees.get(company, min_size)  # Use min_size if company not in degrees
    degree = degrees.get(company, 0)  # Use 0 if company not in degrees
    color = get_company_color(company)
    net.add_node(company, label=company, title=f"{company}\nPartnerships: {degree}", size=size, color=color)


# Add edges to the network
for _, row in df.iterrows():
    edge_color = color_map['Primary nodes'] if row['partner1'] in special_companies and row['partner2'] in special_companies else color_map.get(row['type'], '#FFFFFF')
    net.add_edge(row['partner1'], row['partner2'], title=row['type'], color={'color': edge_color, 'opacity': 0.4})


# Set physics layout
#net.barnes_hut(gravity=-10000, central_gravity=0.3, spring_length=200, spring_strength=0.05, damping=0.09)
#net.barnes_hut(gravity=-15000, central_gravity=0.2, spring_length=300, spring_strength=0.04, damping=0.09)
#net.barnes_hut(gravity=-20000, central_gravity=0.1, spring_length=400, spring_strength=0.04, damping=0.09)
#net.repulsion(node_distance=200, central_gravity=0.1, spring_length=200, spring_strength=0.04, damping=0.09)
net.force_atlas_2based(gravity=-50, central_gravity=0.01, spring_length=100, spring_strength=0.08, damping=0.4, overlap=0)


# show buttons
net.show_buttons()

# Generate the HTML file
#net.show("AI strategic partnerships.html")
net.show("index.html")

index.html


In [13]:
# Add a legend for partnership types
legend_html = "<div style='position:absolute; top:300px; left:10px; background-color:rgba(255,255,255,0.7); padding:10px; border-radius:5px;'>"
legend_html += "<h3>Partnership Types</h3>"
for partnership, color in color_map.items():
    legend_html += f"<div><span style='display:inline-block; width:20px; height:20px; background-color:{color};'></span> {partnership}</div>"
legend_html += "</div>"


# Generate the graph with legend
net.show("index.html", notebook=False)

# Add legend to the generated HTML file
with open("index.html", "r") as file:
    content = file.read()
    content = content.replace("</body>", f"{legend_html}</body>")

with open("index.html", "w") as file:
    file.write(content)

index.html
